In [25]:
!pip install scikit-learn 
!pip install imbalanced-learn


In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [4]:


df = pd.read_excel("Patients Data ( Used for Heart Disease Prediction ).xlsx")
df.head()


,PatientID,State,Sex,GeneralHealth,AgeCategory,HeightInMeters,WeightInKilograms,BMI,HadHeartAttack,HadAngina,...,ECigaretteUsage,ChestScan,RaceEthnicityCategory,AlcoholDrinkers,HIVTesting,FluVaxLast12,PneumoVaxEver,TetanusLast10Tdap,HighRiskLastYear,CovidPos
0,1,Alabama,Female,Fair,Age 75 to 79,1.63,84.820000,32.099998,0,1,...,Never used e-cigarettes in my entire life,1,"White only, Non-Hispanic",0,0,0,1,"No, did not receive any tetanus shot in the pa...",0,1
1,2,Alabama,Female,Very good,Age 65 to 69,1.60,71.669998,27.990000,0,0,...,Never used e-cigarettes in my entire life,0,"White only, Non-Hispanic",0,0,1,1,"Yes, received Tdap",0,0
2,3,Alabama,Male,Excellent,Age 60 to 64,1.78,71.209999,22.530001,0,0,...,Never used e-cigarettes in my entire life,0,"White only, Non-Hispanic",1,0,0,0,"Yes, received tetanus shot but not sure what type",0,0
3,4,Alabama,Male,Very good,Age 70 to 74,1.78,95.250000,30.129999,0,0,...,Never used e-cigarettes in my entire life,0,"White only, Non-Hispanic",0,0,1,1,"Yes, received tetanus shot but not sure what type",0,0
4,5,Alabama,Female,Good,Age 50 to 54,1.68,78.019997,27.760000,0,0,...,Never used e-cigarettes in my entire life,1,"Black only, Non-Hispanic",0,0,1,0,"No, did not receive any tetanus shot in the pa...",0,0


Columnas para el one hot encoding: State, 

In [10]:
# Data Cleaning Function

for col in df.columns:
    print(f"{col}: {df[col].value_counts()}\n")

PatientID: PatientID
1         1
2         1
3         1
4         1
5         1
         ..
237626    1
237627    1
237628    1
237629    1
237630    1
Name: count, Length: 237630, dtype: int64

State: State
Washington              14241
Maryland                 8817
Minnesota                8712
Ohio                     8700
New York                 8625
Texas                    7267
Florida                  7124
Kansas                   6000
Wisconsin                5890
Maine                    5709
Iowa                     5492
Indiana                  5393
South Carolina           5360
Virginia                 5358
Arizona                  5302
Hawaii                   5262
Utah                     5212
Michigan                 5206
Massachusetts            5164
Nebraska                 5008
Colorado                 4973
Georgia                  4860
California               4801
Connecticut              4765
Vermont                  4569
South Dakota             4280
Montana    

In [7]:
#Auxiliary Functions
dummy_cols = ['Sex', 'SmokerStatus', 'ECigaretteUsage', 'RaceEthnicityCategory']
health_map = {
    "Poor": 0,
    "Fair": 1,
    "Good": 2,
    "Very good": 3,
    "Excellent": 4
}

age_map = {
    "Age 18 to 24": 0,
    "Age 25 to 29": 1,
    "Age 30 to 34": 2,
    "Age 35 to 39": 3,
    "Age 40 to 44": 4,
    "Age 45 to 49": 5,
    "Age 50 to 54": 6,
    "Age 55 to 59": 7,
    "Age 60 to 64": 8,
    "Age 65 to 69": 9,
    "Age 70 to 74": 10,
    "Age 75 to 79": 11,
    "Age 80 or older": 12
}

In [22]:
def clean_data(df):
    df_clean = df.copy()

    
    # ------------------------------
    # 1. Drop irrelevant identifiers
    # ------------------------------
    df_clean = df_clean.drop(columns=["PatientID", "State"])

    # ------------------------------
    # 2. Ordinal encode
    # ------------------------------
    df_clean["GeneralHealth"] = df_clean["GeneralHealth"].map(health_map)
    df_clean["AgeCategory"] = df_clean["AgeCategory"].map(age_map)
    df_clean["Sex"] = df_clean["Sex"].map({"Female": 0, "Male": 1})

    # ------------------------------
    # 3. Diabetes → binary
    # ------------------------------
    df_clean["Diabetes"] = df_clean["HadDiabetes"].replace({
        "No": 0,
        "No, pre-diabetes or borderline diabetes": 1,
        "Yes": 1,
        "Yes, but only during pregnancy (female)": 1
    })
    df_clean = df_clean.drop(columns=["HadDiabetes"])

    # ------------------------------
    # 4. Tetanus → binary
    # ------------------------------
    df_clean["TetanusStatus"] = df_clean["TetanusLast10Tdap"].replace({
        "No, did not receive any tetanus shot in the past 10 years": 0,
        "Yes, received tetanus shot but not sure what type": 1,
        "Yes, received Tdap": 1,
        "Yes, received tetanus shot, but not Tdap": 1
    })
    df_clean = df_clean.drop(columns=["TetanusLast10Tdap"])

    # ------------------------------
    # 5. Dummy encoding LOW-cardinality categories
    # ------------------------------
    dummy_cols = [
        "SmokerStatus",
        "ECigaretteUsage",
        "RaceEthnicityCategory"
    ]

    for col in dummy_cols:
        df_clean = pd.get_dummies(df_clean, columns=[col], drop_first=True)

    # ------------------------------
    # 6. Replace booleans with numeric
    # ------------------------------
    df_clean = df_clean.replace({True: 1, False: 0})

    # ------------------------------
    # 7. Scale ONLY continuous numeric columns
    # ------------------------------
    continuous_cols = ["HeightInMeters", "WeightInKilograms", "BMI"]

    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    df_clean[continuous_cols] = scaler.fit_transform(df_clean[continuous_cols])

    # ------------------------------
    # 8. Drop rare dummy columns (< 50 samples)
    # ------------------------------
    # 8. Drop rare dummy columns (< 50 samples)
    rare_cols = [
    c for c in df_clean.columns 
    if df_clean[c].dtype != "O"            # not object (string)
    and df_clean[c].nunique() == 2         # only dummy/binary columns
    and df_clean[c].sum() < 50             # rare positive samples
]

    df_clean = df_clean.drop(columns=rare_cols)

    # ------------------------------
    # 9. Final checks
    # ------------------------------
    if df_clean.isna().sum().sum() != 0:
        print("⚠ NaNs detected!! Check mapping errors.")

    # ------------------------------
    # 10. Sort columns for stable MLP input
    # ------------------------------
    df_clean = df_clean.sort_index(axis=1)

    return df_clean


In [26]:
target = "HadHeartAttack"
df_cleaned = clean_data(df)
df_cleaned.head()

from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(df_cleaned.drop(target, axis=1), df_cleaned[target])


C:\Users\Ale\AppData\Local\Temp\ipykernel_4744\3257465801.py:20: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean["Diabetes"] = df_clean["HadDiabetes"].replace({
C:\Users\Ale\AppData\Local\Temp\ipykernel_4744\3257465801.py:31: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_clean["TetanusStatus"] = df_clean["TetanusLast10Tdap"].replace({
C:\Users\Ale\AppData\Local\Temp\ipykernel_4744\3257465801.py:54: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old be

In [16]:
print(df_cleaned.dtypes)


GeneralHealth                                          float64
AgeCategory                                            float64
HeightInMeters                                         float64
WeightInKilograms                                      float64
BMI                                                    float64
HadHeartAttack                                           int64
HadAngina                                              float64
HadStroke                                              float64
HadAsthma                                              float64
HadSkinCancer                                          float64
HadCOPD                                                float64
HadDepressiveDisorder                                  float64
HadKidneyDisease                                       float64
HadArthritis                                           float64
DeafOrHardOfHearing                                    float64
BlindOrVisionDifficulty                                

In [27]:
#Export cleaned data
df_cleaned.to_csv("cleaned_patient_data.csv", index=False)